# Lab Instructions

You have been hired by a forensic science team to help develop a model that they hope can be used to predict the length of time a body has been buried based on the amount of nitrogen remaining in the femur of the skeleton.  You can download the data using [this link](https://github.com/Chelsea-Myers/Datasets/blob/master/Longbones.csv).  

A description of the dataset follows:

Nitrogen content among 42 male skeletons from
2 cemeteries of various interrnment lengths, ages, depths, lime addition.
Some missing ages and depths. The first 3 cases were contaminated
with oil.

| Variables | Values |
| :---------------- | ------: | 
|Burial Site  |1=A, 2=LG   |
|Deposition (internment) Time  | Years |
|Depth |feet|
|Lime Added | 1=Yes, 0=No |
|Age of Person | Years
|Nitrogen Level | g/100g of bone |
|Oil Contamination | 1=Yes, 0=No |


Source: D.R. Jarvis (1997). "Nitrogen Levels in Long Bones from
Coffin Burials Interred for Periods of 26-90 Years," Forensic Science
International, Vol85, pp199-208.


First, determine if it makes sense to model the relationship between nitrogen level and deposition (interment) time.  Justify your answer.  If it does, then determine how each of the other features (burial site, depth, lime added, age of person, and oil contamination) affect the relationship (if at all).  

If a new skeleton with an unknown interment time was discovered in one of the cemeteries, what information should the forensic scientists collect in order to predict how long it has been buried?